# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [3]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [4]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [5]:
#Definition of the tables.
import pandas as pd
# Table and definitions sample
data = {
    'table': ['employees', 'salary', 'studies'],
    'definition': [
        'Employee information, name, salary, department, position',
        'Salary details for each year, salary, year, bonus, deductions',
        'Educational studies, name of institution, GPA score, degree, year of graduation'
    ]
}
df = pd.DataFrame(data)
print(df)

       table                                         definition
0  employees  Employee information, name, salary, department...
1     salary  Salary details for each year, salary, year, bo...
2    studies  Educational studies, name of institution, GPA ...


In [6]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [7]:
print(text_tables)

employees: Employee information, name, salary, department, position
salary: Salary details for each year, salary, year, bonus, deductions
studies: Educational studies, name of institution, GPA score, degree, year of graduation


In [15]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Important Notes:
- Educational degree information is stored in the 'studies' table
- Salary and compensation details are in 'salary' table
- Queries about degrees or education require the 'studies' table

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [16]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="What is the average salary of all employees?")

In [14]:
print(return_OAI(pqt1))

```json
["employees", "salary"]
```


In [17]:
# Example 2: Education-related query
pqt2 = prompt_question_tables.format(
    tables=text_tables,
    question="Which employees have a GPA above 3.5?"
)
print(return_OAI(pqt2))


```json
{
    "tables": ["employees", "studies"]
}
```


In [18]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="What is the average salary of employees who have a master's degree?")

In [19]:
print(return_OAI(pqt3))

{
    "tables": ["employees", "salary", "studies"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [20]:
#Creating the prompt, with the user questions and the tables definitions.
# Example 3: Complex query involving all tables
pqt4 = prompt_question_tables.format(
    tables=text_tables,
    question="What is the average bonus for employees with a Computer Science degree who make above $100,000?"
)
print("Test Case 3 - Complex Query:")
print(return_OAI(pqt4))

# Example 4: Edge case with ambiguous salary reference
pqt5 = prompt_question_tables.format(
    tables=text_tables,
    question="List all employees who had a salary increase of more than 10% between 2022 and 2023"
)
print("\nTest Case 4 - Temporal Salary Analysis:")
print(return_OAI(pqt5))

# Example 5: Query with potentially missing information
pqt6 = prompt_question_tables.format(
    tables=text_tables,
    question="Show me the department breakdown of employees with PhD degrees and their total bonuses"
)
print("\nTest Case 5 - Department and Education Query:")
print(return_OAI(pqt6))


Test Case 3 - Complex Query:
```json
["employees", "salary", "studies"]
```

Test Case 4 - Temporal Salary Analysis:
{
    "tables": ["employees", "salary"]
}

Test Case 5 - Department and Education Query:
{
    "tables": ["employees", "studies", "salary"]
}


## Analysis Report

### Overview of GPT's Table Selection Performance

#### Initial vs Improved Prompting
The study revealed that the quality of prompts significantly impacts GPT's ability to identify correct table relationships. After adding explicit relationship guidance in the prompt, the model's performance improved notably.

#### Test Cases and Results
1. **Basic Queries**
   - "What is the average salary of all employees?"
   - Tables selected: employees, salary
   - Result: Correct selection with both prompts

2. **Education-Only Queries**
   - "Which employees have a GPA above 3.5?"
   - Tables selected: employees, studies
   - Result: Correct selection with both prompts

3. **Complex Joined Queries**
   - "What is the average salary of employees who have a master's degree?"
   - Initial Result:  Missed 'studies' table (only selected employees, salary)
   - Improved Result: Correctly identified all three tables needed

4. **Advanced Complex Queries**
   - "Average bonus for employees with Computer Science degree making above $100,000"
   - Result: Correctly identified all three tables needed
   - Shows improvement in handling multi-condition queries

5. **Temporal Analysis**
   - "Salary increases > 10% between 2022-2023"
   - Result: Correctly identified employees and salary tables
   - Demonstrates understanding of historical data requirements

### Key Findings

#### What Worked Well
1. **Explicit Relationship Guidance**
   - Adding clear table relationship rules significantly improved accuracy
   - Model better understood implicit joins needed for complex queries

2. **Simple Queries**
   - Consistently accurate for straightforward single-table queries
   - Reliable for direct two-table relationships

3. **Pattern Recognition**
   - Successfully identified salary-related patterns requiring both employees and salary tables
   - Recognized education-related queries needing studies table access

#### Areas for Improvement
1. **Complex Relationships**
   - Initial prompt struggled with queries requiring three-table joins
   - Required explicit guidance about table relationships

2. **Query Context**
   - Model sometimes missed implicit requirements without clear prompting
   - Benefits from explicit rules about table usage

### Lessons Learned
1. **Prompt Engineering Matters**
   - Clear relationship guidelines significantly improve accuracy
   - Explicit rules help prevent missing necessary tables

2. **Model Capabilities**
   - GPT-3.5 can reliably identify table requirements when properly guided
   - Performance depends heavily on prompt quality and explicitness

3. **Best Practices**
   - Include relationship rules in prompts
   - Specify common query patterns
   - Make implicit relationships explicit

### Recommendations
1. Always include table relationship guidelines in prompts
2. Specify common query patterns that require multiple tables
3. Test complex queries to verify relationship understanding
4. Consider adding metadata about foreign key relationships
5. Regular validation of results for complex queries